## Techniche - Author-topic model

In [34]:
import pandas as pd
import numpy as np

import gensim
import gensim.corpora as corpora
from gensim.corpora import mmcorpus
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, Phraser
from gensim.models.ldamodel import LdaModel
from gensim.models import AuthorTopicModel
from gensim.test.utils import common_dictionary, datapath, temporary_file
from smart_open import smart_open

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, punkt, RegexpTokenizer, wordpunct_tokenize
from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer

import json
from pandas.io.json import json_normalize
import requests
import re
import os
import calendar

from topic_model import tokenize_docs, get_patents_by_month # TODO (Lee) resolve

from smart_open import smart_open

import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim

from pprint import pprint

In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
np.random.seed(3)

### Import Data

#### Import data from PatentsView API

In [ ]:
# TODO (Lee) - functionize/comment out, or clarify api call for user to obtain patent_fields list

# build list of possible fields that endpoint request will return
df_api_fields = pd.read_excel("data/patents_view_patents_fields.xlsx")
df_api_fields.columns = (df_api_fields.columns.str.strip()
                                     .str.lower().str.replace(' ', '_')
                                     .str.replace('(', '').str.replace(')', ''))
pat_fields = df_api_fields.api_field_name.values.tolist()
pprint(pat_fields)

In [36]:
%%capture 
# TODO (LEE) - why printing to output?
data = get_patents_by_month("2018-01-01", "2018-07-01",pats_per_page=1000);

In [49]:
def get_patents_by_month(begin_date,end_date, pats_per_page):
    """ requests patent data from PatentsView API by date range"""
    import json
    import requests
    endpoint_url = 'http://www.patentsview.org/api/patents/query'
    page_counter=1
    data = []
    # results = {}
    count=1
    pat_fields= ['assignee_city',
    'assignee_country',
    'assignee_county',
    'assignee_county_fips',
    'assignee_first_name',
    'assignee_first_seen_date',
    'assignee_id',
    'assignee_last_name',
    'assignee_last_seen_date',
    'assignee_lastknown_city',
    'assignee_lastknown_country',
    'assignee_lastknown_latitude',
    'assignee_lastknown_location_id',
    'assignee_lastknown_longitude',
    'assignee_lastknown_state',
    'assignee_latitude',
    'assignee_location_id',
    'assignee_longitude',
    'assignee_organization',
    'assignee_sequence',
    'assignee_state',
    'assignee_state_fips',
    'assignee_total_num_inventors',
    'assignee_total_num_patents',
    'assignee_type',
    'cpc_category',
    'cpc_first_seen_date',
    'cpc_group_id',
    'cpc_group_title',
    'cpc_last_seen_date',
    'cpc_section_id',
    'cpc_sequence',
    'cpc_subgroup_id',
    'cpc_subgroup_title',
    'cpc_subsection_id',
    'cpc_subsection_title',
    'cpc_total_num_assignees',
    'cpc_total_num_inventors',
    'cpc_total_num_patents',
    'detail_desc_length',
    'forprior_country',
    'forprior_date',
    'forprior_docnumber',
    'forprior_kind',
    'forprior_sequence',
    'inventor_city',
    'inventor_country',
    'inventor_county',
    'inventor_county_fips',
    'inventor_first_name',
    'inventor_first_seen_date',
    'inventor_id',
    'inventor_last_name',
    'inventor_last_seen_date',
    'inventor_lastknown_city',
    'inventor_lastknown_country',
    'inventor_lastknown_latitude',
    'inventor_lastknown_location_id',
    'inventor_lastknown_longitude',
    'inventor_lastknown_state',
    'inventor_latitude',
    'inventor_location_id',
    'inventor_longitude',
    'inventor_sequence',
    'inventor_state',
    'inventor_state_fips',
    'inventor_total_num_patents',
    'lawyer_first_name',
    'lawyer_first_seen_date',
    'lawyer_id',
    'lawyer_last_name',
    'lawyer_last_seen_date',
    'lawyer_organization',
    'lawyer_sequence',
    'lawyer_total_num_assignees',
    'lawyer_total_num_inventors',
    'lawyer_total_num_patents',
    'nber_category_id',
    'nber_category_title',
    'nber_first_seen_date',
    'nber_last_seen_date',
    'nber_subcategory_id',
    'nber_subcategory_title',
    'nber_total_num_assignees',
    'nber_total_num_inventors',
    'nber_total_num_patents',
    'patent_abstract',
    'patent_date',
    'patent_firstnamed_assignee_city',
    'patent_firstnamed_assignee_country',
    'patent_firstnamed_assignee_id',
    'patent_firstnamed_assignee_latitude',
    'patent_firstnamed_assignee_location_id',
    'patent_firstnamed_assignee_longitude',
    'patent_firstnamed_assignee_state',
    'patent_firstnamed_inventor_city',
    'patent_firstnamed_inventor_country',
    'patent_firstnamed_inventor_id',
    'patent_firstnamed_inventor_latitude',
    'patent_firstnamed_inventor_location_id',
    'patent_firstnamed_inventor_longitude',
    'patent_firstnamed_inventor_state',
    'patent_kind',
    'patent_number',
    'patent_processing_time',
    'patent_title',
    'patent_type',
    'patent_year',
    'pct_102_date',
    'pct_371_date',
    'pct_date',
    'pct_docnumber',
    'pct_doctype',
    'pct_kind',
    'rawinventor_first_name',
    'rawinventor_last_name',
    'wipo_field_id',
    'wipo_field_title',
    'wipo_sector_title',
    'wipo_sequence']
    while True: # TODO (Lee) - replace with datetime for begin_date to end_date
        
        if count ==0:
            print("error/complete")
            break
            
        elif count > 0:     
            # build query
            query = {"_and":[{"_gte":{"patent_date":begin_date}},{"_lte":{end_date:"2019-01-01"}}]},
            query={"_or":[{"_text_phrase":{"patent_title":"natural language"}},
                          {"_text_phrase":{"patent_abstract":"natural language"}}]}
#             {"_or":[{"cpc_subgroup_id": "G06T3/4046"},
#                     {"cpc_subgroup_id": "G06T9/002"}]}
            fields=pat_fields
            options={"page": page_counter, "per_page":pats_per_page}
            sort=[{"patent_date":"desc"}]
            params={'q': json.dumps(query),
                    'f': json.dumps(fields),
                    'o': json.dumps(options),
                    's': json.dumps(sort)
                        }
    
            # request and results
            response = requests.get(endpoint_url, params=params)
            status = response.status_code
            # print("status:", status,';',"page_counter:",page_counter,) # ";", "iteration:",i
            results = response.json()
            # extract data from response
            data_response = results['patents']
            count = results["count"]
            total_pats = results.get("total_patent_count")
            # print("patents on current page:",count,';', "total patents:",total_pats)
            data.append(data_response)
            page_counter+=1
        
        else:
            print("error #2/complete")
            break
        
    return data
            # TODO (Lee) ? results =  json.loads(response.content)
            # TODO (Lee) ? places.extend(results['results'])
            # TODO (Lee) ? time.sleep(2)

In [53]:
%%capture 
# TODO (LEE) - why printing to output?
data = get_patents_by_month("2018-01-01", "2018-01-31",pats_per_page=1000);

In [60]:
len(data)

4

In [59]:
len(data[0])

1000

In [62]:
raw_df =pd.DataFrame(data[0])
raw_df

,assignees,cpcs,detail_desc_length,foreign_priority,inventors,lawyers,nbers,patent_abstract,patent_date,patent_firstnamed_assignee_city,...,patent_firstnamed_inventor_state,patent_kind,patent_number,patent_processing_time,patent_title,patent_type,patent_year,pct_data,rawinventors,wipos
0,"[{'assignee_city': 'Burlington', 'assignee_cou...","[{'cpc_category': None, 'cpc_first_seen_date':...",11570,"[{'forprior_country': None, 'forprior_date': N...","[{'inventor_city': 'Newton', 'inventor_country...","[{'lawyer_first_name': None, 'lawyer_first_see...","[{'nber_category_id': None, 'nber_category_tit...",Designing a natural language understanding (NL...,2019-03-12,Burlington,...,MA,B2,10229106,2055,Initializing a workspace for building a natura...,utility,2019,"[{'pct_102_date': None, 'pct_371_date': None, ...","[{'rawinventor_first_name': 'Jeffrey N.', 'raw...","[{'wipo_field_id': None, 'wipo_field_title': N..."
1,"[{'assignee_city': 'Mountain View', 'assignee_...","[{'cpc_category': None, 'cpc_first_seen_date':...",28118,"[{'forprior_country': None, 'forprior_date': N...","[{'inventor_city': 'Adliswil', 'inventor_count...","[{'lawyer_first_name': None, 'lawyer_first_see...","[{'nber_category_id': None, 'nber_category_tit...","Methods, systems, and apparatus, including com...",2019-03-12,Mountain View,...,None,B1,10229109,547,Allowing spelling of arbitrary words,utility,2019,"[{'pct_102_date': None, 'pct_371_date': None, ...","[{'rawinventor_first_name': 'Evgeny A.', 'rawi...","[{'wipo_field_id': None, 'wipo_field_title': N..."
2,"[{'assignee_city': 'Seattle', 'assignee_countr...","[{'cpc_category': None, 'cpc_first_seen_date':...",119654,"[{'forprior_country': None, 'forprior_date': N...","[{'inventor_city': 'Seattle', 'inventor_countr...","[{'lawyer_first_name': None, 'lawyer_first_see...","[{'nber_category_id': None, 'nber_category_tit...",A content management system (CMS) and a transl...,2019-03-12,Seattle,...,WA,B1,10229113,895,Leveraging content dimensions during the trans...,utility,2019,"[{'pct_102_date': None, 'pct_371_date': None, ...","[{'rawinventor_first_name': 'Thibault Pierre',...","[{'wipo_field_id': None, 'wipo_field_title': N..."
3,"[{'assignee_city': 'Armonk', 'assignee_country...","[{'cpc_category': None, 'cpc_first_seen_date':...",39588,"[{'forprior_country': None, 'forprior_date': N...","[{'inventor_city': None, 'inventor_country': N...","[{'lawyer_first_name': 'William J.', 'lawyer_f...","[{'nber_category_id': None, 'nber_category_tit...",An approach is provided in which a knowledge m...,2019-03-12,Armonk,...,MN,B2,10229156,1590,Using priority scores for iterative precision ...,utility,2019,"[{'pct_102_date': None, 'pct_371_date': None, ...","[{'rawinventor_first_name': 'Nirav P.', 'rawin...","[{'wipo_field_id': None, 'wipo_field_title': N..."
4,"[{'assignee_city': 'Mt. View', 'assignee_count...","[{'cpc_category': None, 'cpc_first_seen_date':...",55755,"[{'forprior_country': None, 'forprior_date': N...","[{'inventor_city': 'New York', 'inventor_count...","[{'lawyer_first_name': None, 'lawyer_first_see...","[{'nber_category_id': None, 'nber_category_tit...",Computer-implemented systems and methods are p...,2019-03-12,Mt. View,...,NY,B1,10229173,775,Systems and methods for generating responses t...,utility,2019,"[{'pct_102_date': None, 'pct_371_date': None, ...","[{'rawinventor_first_name': 'Engin Cinar', 'ra...","[{'wipo_field_id': None, 'wipo_field_title': N..."
5,"[{'assignee_city': 'Armonk', 'assignee_country...","[{'cpc_category': None, 'cpc_first_seen_date':...",46741,"[{'forprior_country': None, 'forprior_date': N...","[{'inventor_city': 'New Bedford', 'inventor_co...","[{'lawyer_first_name': None, 'lawyer_first_see...","[{'nber_category_id': None, 'nber_category_tit...",Aspects include determination of automated res...,2019-03-12,Armonk,...,MA,B2,10229187,1023,System for determination of automated response...,utility,2019,"[{'pct_102_date': None, 'pct_371_date': None, ...","[{'rawinventor_first_name':

#### Structure data

In [13]:
# extract metadata from response
print("status code:", response.status_code,';', "reason:", response.reason)
total_patent_count = results["total_patent_count"]
patents_per_page = results['count']
print("total_patent_count:",total_patent_count,';', "patents_per_page:", patents_per_page)

# extract data from response
data_response = results['patents']
# data_resp[0]

NameError: name 'response' is not defined

#### Subset dataframe

In [48]:
raw_df = pd.DataFrame(data)

TypeError: object of type 'NoneType' has no len()

In [15]:
raw_df.head(10)

,count,patents,total_patent_count
0,1000,"[{'detail_desc_length': '10031', 'patent_abstr...",30873
1,1000,"[{'detail_desc_length': '4401', 'patent_abstra...",30873
2,1000,"[{'detail_desc_length': '33335', 'patent_abstr...",30873
3,1000,"[{'detail_desc_length': '21324', 'patent_abstr...",30873
4,1000,"[{'detail_desc_length': '28665', 'patent_abstr...",30873
5,1000,"[{'detail_desc_length': '38775', 'patent_abstr...",30873
6,1000,"[{'detail_desc_length': '51689', 'patent_abstr...",30873
7,1000,"[{'detail_desc_length': '10752', 'patent_abstr...",30873
8,1000,"[{'detail_desc_length': '7910', 'patent_abstra...",30873
9,1000,"[{'detail_desc_length': '20981', 'patent_abstr...",30873


In [ ]:
# subset dataframe - comment/uncomment to include fields
df = raw_df[['patent_number', 'patent_date', 'patent_title',
             'patent_abstract', 'patent_firstnamed_assignee_id',
             'patent_firstnamed_assignee_location_id',
             'patent_firstnamed_assignee_latitude',
             'patent_firstnamed_assignee_longitude',
             'patent_firstnamed_assignee_city',
             'patent_firstnamed_assignee_state',
             'patent_firstnamed_assignee_country', 
             'patent_firstnamed_inventor_id',
             'patent_firstnamed_inventor_location_id',
             'patent_firstnamed_inventor_latitude',
             'patent_firstnamed_inventor_longitude',
             'patent_firstnamed_inventor_city',
             'patent_firstnamed_inventor_state',
             'patent_firstnamed_inventor_country',
             'patent_year', 'patent_type', 'patent_kind',
             'inventors'
            ]]

#### Explore data

In [ ]:
# 561 different assignees
len(df.patent_firstnamed_assignee_id.unique())

#### Create new column

In [ ]:
# create new column that combines the patent title and the patent abstract columns into a single string
df['patent_title_abstract'] = df.patent_title + ' ' + df.patent_abstract
df.patent_title_abstract.head(3)

In [ ]:
# TODO (Lee) partition data
len(text_data)
text_train = text_data[:round(len(text_data)*.8)]
text_test = text_data[round(len(text_data)*.8):]
print(len(text_data), len(text_train), len(text_test), len(text_train)+len(text_test))

In [ ]:
# sort values in dataframe
df.sort_values(by=['patent_date'])

In [ ]:
# convert dataframe to list
text_data = df.patent_title_abstract.tolist()
text_data

### Pre-process text data

In [ ]:
# uncomment to download stop words from nltk and language package from spacy
# nltk.download('stopwords')
# nltk.download('punkt')
# !python -m spacy download en

In [ ]:
# construct pipeline using Spacy Language object and associated pipeline/components
nlp = spacy.load("en")
pprint(nlp.pipeline)

In [ ]:
processed_docs = []   

# process patent documents in pipeline
for doc in nlp.pipe(text_train, n_threads=4, batch_size=100):
   
    ents = doc.ents  # Named entities.

    # Keep only words (no numbers, no punctuation).
    # Lemmatize tokens, remove punctuation and remove stopwords.
    doc = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

    # Remove common words from a stopword list.
    doc = [token for token in doc if token not in stop_words]

    # Add named entities, but only if they are a compound of more than word.
    doc.extend([str(entity) for entity in ents if len(entity) > 1])
    
    processed_docs.append(doc)

processed_docs[0][:5]

In [ ]:
nlp.

In [ ]:
[token.text for token in doc]

In [ ]:
labels = set([w.label_ for w in doc.ents]) 

In [ ]:
for label in labels: 
    entities = [cleanup(e.string, lower=False) for e in document.ents if label==e.label_] 
    entities = list(set(entities)) 
    print(label,entities)

In [ ]:
pre_processed_docs = []
for doc in nlp.pipe(docs, n_threads=4, batch_size=100):
    # Process document using Spacy NLP pipeline.
    
    ents = doc.ents  # Named entities.

    # Keep only words (no numbers, no punctuation).
    # Lemmatize tokens, remove punctuation and remove stopwords.
    doc = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

    # Remove common words from a stopword list.
    #doc = [token for token in doc if token not in STOPWORDS]

    # Add named entities, but only if they are a compound of more than word.
    doc.extend([str(entity) for entity in ents if len(entity) > 1])
    
    pre_processed_docs.append(doc)

#### Tokenize

In [ ]:
# tokenize documents

def tokenize_docs(docs):
    tokenized_docs = []
    for doc in docs:
        tokenized_docs.append(word_tokenize(doc))
    return tokenized_docs

tokenized_docs = tokenize_docs(text_train)

#### Clean punctuation

In [ ]:
# clean punctuation
def clean_docs(tokenized_docs):
    clean_docs = []
    for doc in tokenized_docs:
       clean_docs.append([word for word in doc if word.isalpha()])  
    return clean_docs

In [ ]:
cleaned_data = clean_docs(tokenized_docs)
cleaned_data[0]

#### Convert to lowercase

In [ ]:
# convert to lowercase
def lower_words(docs):
    lowered_words = []
    for doc in docs:
        lowered_words.append([word.lower() for word in doc])
    return lowered_words

lowered_data = lower_words(cleaned_data)
lowered_data[0]

#### Clean stopwords

In [ ]:
# clean stopwords

stop_words = stopwords.words('english')

In [ ]:
def filter_stopwords(docs):
    filtered_docs = []
    for doc in docs:
       filtered_docs.append([word for word in doc if word not in stop_words])
    return filtered_docs

# remove stopwords
filtered_data = filter_stopwords(lowered_data)
filtered_data
# TODO (Lee) - resolve un-lowered stopwords "A" and "An", 'By', 'The'

#### Construct bigrams and trigrams

In [ ]:
# train bigram phrases model
bigram_model = Phrases(filtered_data, min_count=1, threshold=1)

# train trigram phrases model
trigram_model = Phrases(bigram_model[filtered_data], threshold=100)  

In [ ]:
# bigrams
def bigrams(docs):
    """create bigrams"""
    return [bigram_model[doc] for doc in docs]

In [ ]:
# initialize bigram and trigram models
bigram_model = gensim.models.phrases.Phraser(bigram_model)
trigram_model = gensim.models.phrases.Phraser(trigram_model)

In [ ]:
bigrams(filtered_data)[0]

In [ ]:
def trigrams(docs):
    """create trigrams"""
    return [trigram_model[bigram_model[doc]] for doc in docs]

In [ ]:
trigrams(filtered_data)[0]

#### Stem and Lemmatize

In [ ]:
def lemmatize_docs(docs, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """lemmatize documents"""
    lemmatized_docs = []
    for doc in docs: 
        lemmatized_docs.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return lemmatized_docs

In [ ]:
# TODO (Lee)

lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)
# for doc in cleaned_data:
#     for token in doc:
#         token.lemma_

# uncomment to use
# download english model with "python -m spacy download en"

# for token in doc:
#     print(token, token.lemma, token.lemma_)

# TODO (Lee) - lemmatize_docs(cleaned_data)

#### Create corpus and dictionary

In [ ]:
# using spacy pipeline components
# build dictionary
id_to_word = corpora.Dictionary(processed_docs)

# build corpus
texts = processed_docs

# apply term document frequency
# converts documents in corpus to bag-of-words format, a list of (token_id, token_count) tuples
corpus = [id_to_word.doc2bow(doc) for doc in processed_docs]

In [ ]:
 # build dictionary
id_to_word = corpora.Dictionary(filtered_data)

# build corpus
texts = filtered_data

# apply term document frequency
# converts documents in corpus to bag-of-words format, a list of (token_id, token_count) tuples
corpus = [id_to_word.doc2bow(text) for text in texts]

In [ ]:
# view formatted corpus (term-doc-frequency)
[[(id_to_word[id], freq) for id, freq in text] for text in corpus][:1]

### Model - model #1

In [ ]:
# TODO (Lee) - deprecation warnings
# construct LDA model
model_lda = LdaModel(corpus=corpus,
                     id2word=id_to_word,
                     num_topics=25, 
                     random_state=100,
                     update_every=1,
                     chunksize=100,
                     passes=10,
                     alpha='auto',
                     per_word_topics=True)

In [ ]:
# print keywords in n topics
pprint(model_lda.print_topics())

In [ ]:
# print top 10 keywords that comprise topic with index of 0
pprint(model_lda.print_topic(24))
# the most import keywords, and the respective weight, that form topic 0 are

In [ ]:
# print top 10 keywords that comprise topic with index of 1
pprint(model_lda.print_topic(1))

In [ ]:
# TODO (Lee) - infer topic from keywords?

### Evaluate - model #1

In [ ]:
# calculate perplexity metrics
perplexity = model_lda.log_perplexity(corpus)
perplexity

In [ ]:
# TODO (Lee) - confirm that filtered_data is indeed the correct dataset to pass to texts param
# calculate coherence metric
coherence = CoherenceModel(model=model_lda, texts=processed_docs, dictionary=id_to_word, coherence='c_v')
coherence_1 = coherence.get_coherence()
coherence_1

In [ ]:
# TODO (Lee) - confirm that filtered_data is indeed the correct dataset to pass to texts param
# calculate coherence metric
coherence = CoherenceModel(model=model_lda, texts=filtered_docs, dictionary=id_to_word, coherence='c_v')
coherence_1 = coherence.get_coherence()
coherence_1

In [ ]:
# calculate coherence metric or each of the n topicss
coherence_1 = coherence.get_coherence_per_topic()
coherence_1

In [ ]:
# explore topics
pyLDAvis.enable_notebook()
viz_topics_1 = pyLDAvis.gensim.prepare(model_lda, corpus, id_to_word)
viz_topics_1
# TODO (Lee) - salient vs relevant terms in pyLDA ?

### Model 2-  Mallet model

In [ ]:
# uncomment to download Mallet topic model
# !wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
# update this path
path_mallet = 'data/mallet-2.0.8/bin/mallet'

In [ ]:
model_2 = gensim.models.wrappers.LdaMallet(path_mallet, corpus=corpus, num_topics=25, id2word=id_to_word)

In [ ]:
# topics
pprint(model_2.show_topics(formatted=False))

In [ ]:
# calculate coherence metric
coherence_model_2 = CoherenceModel(model=model_2, texts=data, dictionary=id_to_word, coherence='c_v')
coherence_model_2 = coherence_model_2.get_coherence()
coherence_model_2

In [ ]:
# TODO (Lee)
# def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
#     """
#     Compute c_v coherence for various number of topics

#     Parameters:
#     ----------
#     dictionary : Gensim dictionary
#     corpus : Gensim corpus
#     texts : List of input texts
#     limit : Max num of topics

#     Returns:
#     -------
#     model_list : List of LDA topic models
#     coherence_values : Coherence values corresponding to the LDA model with respective number of topics
#     """
#     coherence_values = []
#     model_list = []
#     for num_topics in range(start, limit, step):
#         model = gensim.models.wrappers.LdaMallet(path_mallet, corpus=corpus, num_topics=num_topics, id2word=id2word)
#         model_list.append(model)
#         coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
#         coherence_values.append(coherencemodel.get_coherence())

#     return model_list, coherence_values

# model_list, coherence_values = compute_coherence_values(dictionary=id_to_word, corpus=corpus, texts=data, start=2, limit=40, step=6)

### Model 3 - Author topic model

In [ ]:
# construct inventor-to-doc mapping as df from nested inventors column in json api response
df_inventors = json_normalize(results['patents'], record_path=['inventors'], meta=['patent_number', 'patent_date'])
df_inventors = df_inventors[['inventor_id', 'patent_number', 'patent_date']]
df_inventors.sort_values(by=['patent_date'])
df_inventors.head(3)

In [ ]:
df.head(3)

In [ ]:
# TODO (Lee) - resolve workaround
df_idx = df
df_idx['idx'] = df.index
df_idx
df_idx_1 = df_idx[['patent_number', 'idx', 'inventors']]
df_idx_2 = df_idx_1.set_index('patent_number')
df_idx_2.pop('inventors')
df_idx_2
df_pat_idx = df_idx_2.T.to_dict('records')
for i in df_pat_idx:
    df_pat_idx = dict(i)
df_pat_idx

In [ ]:
df_pat_idx = df_idx_2.T.to_dict('records')
for i in df_pat_idx:
    df_pat_idx = dict(i)
df_pat_idx

In [ ]:
df_inv_test = json_normalize(results['patents'], record_path=['inventors'], meta=['patent_number', 'patent_date'])
df_inv_test.head(3)

In [ ]:
df_idx_pat_inv_map = df[['patent_number', 'inventors']]
df_idx_pat_inv_map.head(3)

In [ ]:
# TODO (Lee) - find out how to get list of patents_view_field names from API - I did it accidentally but need to replicate

In [ ]:
df.patent_title_abstract[0]

In [ ]:
df[:3]

In [ ]:
df_inventors.set_index('inventor_id').T.to_dict('list')

In [ ]:
# for k, v in pat2inv.items():
#     name_dict[new_key] = name_dict.pop(k)
#     time.sleep(4)

# pprint.pprint(name_dict)

# d = {'x':1, 'y':2, 'z':3}
# d1 = {'x':'a', 'y':'b', 'z':'c'}

# dict((d1[key], value) for (key, value) in d.items())
# {'a': 1, 'b': 2, 'c': 3}

In [ ]:
patdf2inv = dict((df_pat_idx[key], value) for (key, value) in pat2inv.items())
patdf2inv

In [ ]:
pat2inv = {k: list(v) for k,v in df_inventors.groupby("patent_number")["inventor_id"]}
pat2inv

In [ ]:
idx_pat_map = df.patent_number.to_dict()
idx_pat_map = {str(key): value for key, value in idx_pat_map.items()}
idx_pat_map

#### Construct author-topic model

In [ ]:
# construct author-topic model
model_at = AuthorTopicModel(corpus=corpus,
                         doc2author=patdf2inv,
                         id2word=id_to_word, 
                         num_topics=25)

In [ ]:
# construct vectors for authors
author_vecs = [model_at.get_author_topics(author) for author in model_at.id2author.values()]
author_vecs

In [ ]:
# retrieve the topic distribution for an author using use model[name] syntax
# each topic has a probability of being expressed given the particular author, but only the ones above a certain threshold are shown.

model_at['7788103-1']

In [ ]:
# def show_author(name):
#     print('\n%s' % name)
#     print('Docs:', model.author2doc[name])
#     print('Topics:')
#     pprint([(topic_labels[topic[0]], topic[1]) for topic in model[name]])

In [ ]:
# calculate per-word bound, which is a measure of the model's predictive performance (reconstruction error?)

build doc2author dictionary

doc2author = atmodel.construct_doc2author(model.corpus, model.author2doc)

In [ ]:
from gensim.models import atmodel
doc2author = atmodel.construct_doc2author(model.corpus, model.author2doc)

In [ ]:
gensim.models.atmodel.construct_author2doc(doc2author)
# construct mapping from author IDs to document IDs.

Parameters:	doc2author (dict of (int, list of str)) – Mapping of document id to authors.
Returns:	Mapping of authors to document ids.
Return type:	dict of (str, list of int)

In [ ]:
gensim.models.atmodel.construct_doc2author(corpus, author2doc)
construct mapping from document IDs to author IDs

Parameters:	
corpus (iterable of list of (int, float)) – Corpus in BoW format.
author2doc (dict of (str, list of int)) – Mapping of authors to documents.
Returns:	
Document to Author mapping.

Return type:	
dict of (int, list of str)